Build a baseline model using the weather station data (this approach used all days in 2018 since the data was available).


Then use the model to make predictions for the ENVI-met(micro-climate) data for that specific day and compare it to model predictions using weather station data also for the same day.

# 1. Import

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import pathlib
import pickle

import pathlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from datetime import datetime
from sklearn import metrics

#  for multicolinearity
from statsmodels.stats.outliers_influence import variance_inflation_factor 

import PyQt5

# 2. Import Files

## 2.1 Save csv files as pickle

Only run this once to save our csv data as pickle files

In [ ]:
# --> NO NEED TO RUN SAVED AS PICKLE FILES <--
# WEATHER FILES ##

# AZ PHX Sky Harbor Data #
AZW_15 = pd.read_csv("./Data/Weather Data/KPHX-15.csv")

# ENVIMET DATA #
BPS = []
Fname = []
for path in pathlib.Path("./Data/BPS").iterdir():
    if path.is_file():
        current_file = pd.read_csv(path)
        BPS.append(current_file)
        Fname.append(path.name.replace('.csv', ''))

# CAMPUS METABOLISM DATA #
metabol14 = []
for path in pathlib.Path('./Data/ASU 2018').iterdir():
    if path.is_file():
        current_file = pd.read_csv(path)
        metabol14.append(current_file)

## Drop last row of EnviMet Data
for i in range(len(BPS)):
    BPS[i] = BPS[i].drop(16)

## Save files as pickle
AZW_15.to_pickle("./Data/AZW_15.pkl")

with open('./Data/BPS.pkl', 'wb') as f:
    pickle.dump(BPS, f)

with open('./Data/Fname.pkl', 'wb') as f:
    pickle.dump(Fname, f)

with open('./Data/metabol14.pkl', 'wb') as f:
    pickle.dump(metabol14, f)

## 2.2 Reload Pickle

In [ ]:
# # 1. Load AZ Weather Data (15-min)
# weather_station = pd.read_pickle('AZW_15.pkl')

# We will convert these to df depending on the building we choose
# 2. Load 14 envimet bldgData (14 filtered buildings)
with open('./Data/BPS.pkl', 'rb') as f:
    envi14 = pickle.load(f)

# 3. Load names of BPS files
with open('./Data/Fname.pkl', 'rb') as f:
    Fname = pickle.load(f)

# 4. Load 14 campus metabolism building energy data
with open('./Data/metabol14.pkl', 'rb') as f:
    metabol14 = pickle.load(f)

## 2.3 Choose building name to import

In [ ]:
##Print Building Names ##
for i in range(len(Fname)):
    print(Fname[i])
    
bldname = input('Enter building name: ')

for i in range(len(envi14)):
    if bldname == Fname[i]:
        # save 
        envi_bldg = envi14[i]

if bldname not in Fname:
    print("\x1b[31m\"Please enter a valid name from the list above\"\x1b[0m")


## 2.4 Create Data Frame for envimet and metabolism data

In [ ]:
class building:
    
    def __init__(self, bldgname):
        self.bldgname = bldgname
    
    def campusmetabolism(self):
        for i in range(len(metabol14)):
            if metabol14[i]['bldgname'][0] == bldname:
                cmp = metabol14[i]
        return cmp
    
    def envimet(self):
        env = envi_bldg[['Date', 'Time', 'AirTempInFrontOfAllFacades_MEAN', 'RelativeAirHumidityInFrontOfAllFacades_MEAN',
                     'WindSpeedInFrontOfAllFacades_MEAN']]
        
        env = env.rename(columns = {'AirTempInFrontOfAllFacades_MEAN':'Air Temp',
                                    'RelativeAirHumidityInFrontOfAllFacades_MEAN':'Rel Humid',
                                    'WindSpeedInFrontOfAllFacades_MEAN':'Wind Speed'})
        return env


Bldg = building(bldname)
metabol = Bldg.campusmetabolism() # campus metabolism
envimet = Bldg.envimet()          # envimet


## 2.5 Create Data Frame for weather station data

In [ ]:
# Load AZ Weather Data (15-min)
weather_station = pd.read_pickle('./Data/AZW_15.pkl')

# 3. Preprocessing

## 3.1 Formatting Date and Time

### a) Envimet dates

In [ ]:
# 1. format time
envimet['Time'] = envimet['Time'].str.replace('.',':')
envimet['Time'] = envimet['Time'].str.replace('01','00')

# 2. convert to 24 hour format
envimet['Time'] = pd.to_datetime(envimet['Time']).dt.strftime('%H:%M')

# 3. format date (still in string)
envimet['Date'] = pd.to_datetime(envimet['Date']).dt.strftime('%m/%d/%Y')

# 4. combine date time column as string and set as index
envimet['Date_Time'] = envimet['Date'] + ' ' + envimet['Time']



In [ ]:
# 4. Get string type for month and time
envimet['Month'] = envimet['Date_Time'].str[0:2]
envimet['Time'] = envimet['Time'].str.replace(':','')

# 5. Rearrange columns
envimet = envimet[['Date_Time','Month','Time', 'Air Temp', 'Rel Humid' ]]

envimet = envimet.set_index('Date_Time')

envimet 

In [ ]:
# # 6. Get numeric for month hour and minute
# microclimate.Date_Time = pd.to_datetime(microclimate.Date_Time)
# microclimate['Month_num'] = microclimate.Date_Time.dt.month
# microclimate['Hour_num'] = microclimate.Date_Time.dt.hour
# microclimate['Minute_num'] = microclimate.Date_Time.dt.minute
# microclimate.Date_Time = pd.to_datetime(microclimate.Date_Time).dt.strftime('%m/%d/%Y %H:%M')

# # 7. Rearrange columns
# print(list(microclimate.columns))
# microclimate = microclimate[['Date_Time','Month','Time','Month_num', 'Hour_num', 'Minute_num', 'Air Temp', 'Rel Humid' ]]

### c) Format weather station dates

In [ ]:
# 1. Get string type for month and time
weather_station['Month'] = weather_station['Date_Time'].str[0:2]
weather_station['Time'] = weather_station['Date_Time'].str[11:]
weather_station['Time'] = weather_station['Time'].str.replace(':','')

# 2. Get numeric for month hour and minute
weather_station.Date_Time = pd.to_datetime(weather_station.Date_Time)
weather_station['Month_num'] = weather_station.Date_Time.dt.month
weather_station['Hour_num'] = weather_station.Date_Time.dt.hour
weather_station['Minute_num'] = weather_station.Date_Time.dt.minute
weather_station.Date_Time = pd.to_datetime(weather_station.Date_Time).dt.strftime('%m/%d/%Y %H:%M')

# 3. set date time as index
weather_station = weather_station.set_index('Date_Time') 

### d) Format Building energy dates

In [ ]:
metabol.tstamp = pd.to_datetime(metabol.tstamp).dt.strftime('%m/%d/%Y %H:%M')

# remove unwanted columns
metabol = metabol[['tstamp','KW', 'CHWTON']]

# set date time as index
metabol = metabol.set_index('tstamp')

metabol

## 3.2 Append Energy Consumption to Weather Station data

In [ ]:
weather_station =  pd.concat([metabol, weather_station], axis = 1, join = "inner")

# rearrange column
weather_station = weather_station[['Month','Time','Month_num', 'Hour_num', 'Minute_num', 'Air Temp', 'Rel Humid', 'KW','CHWTON' ]]
weather_station 

In [ ]:
# weather_station.to_csv('./Data/weather_st')

# 4. Create June 9th Data 

## 4.1 For Microclimate

We want: month, hour, minute, CHWTON, KW, date, air temp, and real humidity for microclimate June 9th.
We do this by merging with building_energy to get KW and CHWTON on the dates that appear in microclimate data



In [ ]:
envimet_j9 = pd.merge(envimet, metabol, left_index = True, right_index = True)
envimet_j9

## 4.2 For Weather Station

In [ ]:
# 1. Extract june 9th data 
station_j9 = weather_station.iloc[15150:15246]

# 2. Filter time ( only minute 00)
station_j9 = station_j9[ (station_j9['Hour_num'] >= 5) & (station_j9['Hour_num'] <= 20) & (station_j9['Minute_num'] == 0)]

# 3. drop June 9th data on original data
weather_station = weather_station.drop(weather_station.index[15150:15246])

# 4. drop numeric column
weather_station = weather_station.drop(labels = ['Hour_num', 'Month_num','Minute_num'], axis = 1)
station_j9 = station_j9.drop(labels = ['Hour_num', 'Month_num','Minute_num'], axis = 1)

In [ ]:
station_j9

In [ ]:
# 5. convert index to date time object
weather_station.index = pd.to_datetime(weather_station.index)

# 6. ensure theres no more june 9th data on weather_station data
print(weather_station[(weather_station.index.month == 6) & (weather_station.index.day == 9)])

# convert back
# weather_station.index = pd.to_datetime(weather_station.index).dt.strftime('%m/%d/%Y %H:%M')

# 5. EDA

## 5.1 Correlation Matrix

In [ ]:
corrMatrix = weather_station.corr()
plt.figure(figsize=(5,5))
sns.heatmap(corrMatrix)
plt.show()

## 5.2 Multicolinearity

A simple method to detect multicollinearity in a model is by using something called the variance inflation factor or the VIF for each predicting variable. An acceptable VIF is if it’s less than the max of 10

In [ ]:
# Get X and Y
Y = weather_station['CHWTON']
X = weather_station.drop(labels = ['CHWTON'], axis = 1)

X_int = X.drop(labels = ['Month', 'Time'], axis = 1)

# get multicolinearity data
vif_data = pd.DataFrame()
vif_data["feature"] = X_int.columns
vif_data["VIF"] = [variance_inflation_factor(X_int.values, i) for i in range(len(X_int.columns))]

vif_data

# 6. Modelling

Scoring:
One one hand, RMSE tells us the typical distance between the predicted value made by the regression model and the actual value.

On the other hand, R2 tells us how well the predictor variables can explain the variation in the response variable.

## 6.1 Train and Predict

In [ ]:
############################ RANDOM FORESTS #################################

# 1. Get train test 
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

# 2. Using RandomForestRegressor to make predictions
from sklearn.ensemble import RandomForestRegressor
# - string
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

# 3. Get prediction
Y_pred = model.predict(X_test)
ModelPred = pd.DataFrame({'Actual CHWTON':Y_test, 'Predicted CHWTON':Y_pred})
ModelPred = ModelPred.sort_index()
print(ModelPred)

## 6.2 Score

In [ ]:
# 1. RMSE and R2
R2_all = model.score(X_test, Y_test)
RMSE_all = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))

# 2.Feature importance
feature_list = list(X_train.columns)
feature_imp = pd.Series(model.feature_importances_, index = feature_list).sort_values(ascending=False)
print("\033[1m" + "Feature Importances:" + "\033[0m")
print(feature_imp, "\n")

# 3. create score df
scores_df = pd.DataFrame({"score_type": ["R2", "RMSE"], "RF_allyear": [R2_all, RMSE_all]})
scores_df

# 7. June 9th Prediction

## 7.1 Weather Station prediction

In [ ]:
# 1. Get X and Y (all test)
X_j9 = station_j9.drop(labels = ['CHWTON'], axis = 1)

# 2. Get Y
Y_actual_j9 = station_j9['CHWTON']

# 3. Get Y_pred
Y_pred_j9 = model.predict(X_j9)

# 4. Score
R2_j9 = model.score(X_j9, Y_actual_j9)
RMSE_j9 = np.sqrt(metrics.mean_squared_error(Y_actual_j9, Y_pred_j9))

# 5. Append score to df
score_j9_AZ = [R2_j9, RMSE_j9]
scores_df['RF_j9_AZ'] = score_j9_AZ

scores_df

## 7.2 Microclimate prediction

In [ ]:
# 1. Get X
X_j9_envi = envimet_j9.drop(labels = ['CHWTON'], axis = 1)

# 2. Get Y_pred
Y_pred_j9_envi = model.predict(X_j9_envi)

# 3. Score
R2_j9_envi = model.score(X_j9_envi, Y_actual_j9)
RMSE_j9_envi = np.sqrt(metrics.mean_squared_error(Y_actual_j9, Y_pred_j9_envi))

# 4. append to score df
score_j9_envi = [R2_j9_envi, RMSE_j9_envi]
scores_df['RF_j9_micro'] = score_j9_envi
scores_df

In [ ]:
Pred = pd.DataFrame({'Actual':Y_actual_j9,'Baseline Predictions (AZW)': Y_pred_j9, 'Microclimate Predictions': Y_pred_j9_envi})
Pred

# 8. Hypothesis Test

In [ ]:
plt.boxplot([Y_pred_j9, Y_pred_j9_envi])
plt.xticks([1,2],['Baseline_pred', 'Microclimate_pred'])

In [ ]:
print(Y_pred_j9.mean())
print(Y_pred_j9_envi.mean())
print('mu:', Y_pred_j9.mean()- Y_pred_j9_envi.mean())

# Two-Sample T Test


mean differences in CHWTON = $ \mu_{baseline} - \mu_{microclimate} $ 

$ H_0: $ Mean of CHWTON in baseline and microclimate are the same

$ H_1: $ Mean of CHWTON in baseline and microclimate are NOT the same

## 8.1 calculate standard deviation

In [ ]:
# 1.variance
var_baseline = Y_pred_j9.var(ddof = 1)
var_micro = Y_pred_j9_envi.var(ddof = 1)
print('var: ',var_baseline, var_micro)

# 2. standard deviation
s = np.sqrt((var_baseline + var_micro)/2)
print('sd: ',s)

## 8.2 calculate T-statistic

In [ ]:
import scipy.stats as stats
t_stat, p_val = stats.ttest_ind(Y_pred_j9, Y_pred_j9_envi, equal_var=False)
print('t statistics: ', t_stat)
print('p value: ', p_val)

P value is not less that 0.05. We cannot reject the null hypothesis. There is no significant difference between the transaction amount of fraud and non fraudulent transaction

# 9. Plot

In [ ]:
####### Plotting Baseline Model for all 2018 15-min Data #######

## This is a big graph, will be slow to run but gives visual of prediction accuracy
# %matplotlib qt
plt.xlabel('Baseline 15-Min Model')
plt.ylabel('CHWTON')
plt.plot(ModelPred['Actual CHWTON'], label = 'Actual CHWT')
plt.plot(ModelPred['Predicted CHWTON'], label = 'Predicted CHWT')
plt.legend()
plt.show()

In [ ]:
#  Plotting ENVI-met vs AZW vs Actual Data for June 9 from 5a - 8p

positions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
labels = ['5a', '6a', '7a', '8a', '9a', '10a', '11a', '12p', '1p', '2p', '3p', '4p', '5p', '6p', '7p', '8p']

# plot EnviMet vs AZ_Weather results
plt.xlabel('Time 5a - 8p')
plt.ylabel('CHWTON')
plt.xticks(positions, labels)
plt.plot(Pred['Microclimate Predictions'], label = 'ENVIMET Weather')
plt.plot(Pred['Baseline Predictions (AZW)'], label = 'Baseline Weather (AZW)')
plt.plot(Pred['Actual'], label = 'Actual Data')
plt.title(bldname)
plt.legend()

## show graphs
plt.show()

# 10. Save DF

## 10.1 Weather Station string


In [ ]:
# weather_station.to_csv('./Data/weather_st2')

## 10.2 June 9th Weather St


In [ ]:
# weather_st_June_9.to_csv('./Data/weather_j9')

In [ ]:
# scores_df.to_csv('./Data/score')